In [22]:
from csv import DictReader

nodes = {}
edges = {}

def create_or_update_node (id, type, value):
    if id not in nodes:
        nodes[id] = {
            "id": id,
            "type": type,
            "weight": value
        }
    else:
        nodes[id]["weight"] += value

def create_or_update_edge (provenance, destination, value):
    footprint = "from " + provenance + " to " + destination
    if footprint not in edges:
        edges[footprint] = {
            "from": provenance,
            "to": destination,
            "weight": value
        }
    else:
        edges[footprint]["weight"] += value

with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        value = int(row['value'].split('.')[0] or 0)
        if \
                row['year'] == '1789' \
            and row['best_guess_region_prodxpart'] == '1' \
            :
            # Marseille is reporting
            if row['customs_region'] == 'Marseille':
                bureau = row['customs_office_source']
                if bureau == "Septeme":
                    bureau = "Septemes"
                if bureau == "Marseille, Poids et Casse":
                    bureau = "Marseille - Poids et Casse"
                if bureau == "Penne":
                    bureau = "La Penne"
                bureau = "bureau des fermes " + bureau
                create_or_update_node(bureau, "Marseille", value)
                if row['export_import'] == 'Exports':
                    if row['partner_grouping'] == 'France':
                        if row['partner_simplification'] == 'Marseille':
                            partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                            create_or_update_node(partner, "Marseille", value)
                        else:
                            partner = "ports francs de France"
                            create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(bureau, partner, value)
                elif row['export_import'] == 'Imports':
                    if row['partner_grouping'] == 'France':  
                        partner = "ports francs de France"
                        create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(partner, bureau, value)
            # Marseille is partner
            if row['partner_simplification'] == 'Marseille':
                reporter = "directions des Fermes hors Marseille"
                create_or_update_node(reporter, "France hors Marseille", value)
                partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                create_or_update_node(partner, "Marseille", value)
                if row['export_import'] == 'Exports':
                    create_or_update_edge(reporter, partner, value)
                else:
                    create_or_update_edge(partner, reporter, value)


In [23]:
import networkx as nx
from ipysigma import Sigma

graph = nx.DiGraph()

for id, props in nodes.items():
    graph.add_node(id)
    for key, val in props.items():
        graph.nodes[id][key] = val

for id, props in edges.items():
    provenance = props["from"]
    destination = props["to"]
    graph.add_edge(provenance, destination)
    for key, val in props.items():
        graph.edges[provenance, destination][key] = val
nx.write_gexf(
    graph, 
    "toflit18-bureaux-network.gexf",
    encoding='utf-8',
    prettyprint=True
)

In [9]:
import json

for_d3 = {
    "nodes": [],
    "links": []
}
sources_indexes = {}
for index, (node_id, node) in enumerate(nodes.items()):
    sources_indexes[node_id] = index

for id, props in nodes.items():
    for_d3["nodes"].append({
        "name": props["id"],
        "label":  props["id"],
        "value": props["weight"],
        "id": id
    })


for id, props in edges.items():
    for_d3["links"].append({
        "source": sources_indexes[props["from"]],
        "target": sources_indexes[props["to"]],
        "value": props["weight"]
    })
with open('toflit18-bureaux-network.json', 'w') as f:
    json.dump(for_d3, f, indent=2)

In [40]:
import networkx as nx
from ipysigma import Sigma

Sigma(graph, node_size="value", node_color='type', background_color="black")

Sigma(nx.DiGraph with 12 nodes and 13 edges)